In [2]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import EEGNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [3]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [3]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 10

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_10_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0253        4.1065       0.0375        3.6888     +  0.0010  0.9497
      2       0.0267        3.8901       0.0338        3.6793     +  0.0010  0.7649
      3       0.0267        3.8330       0.0362        3.6709     +  0.0010  0.7584
      4       0.0290        3.7875       0.0488        3.6613     +  0.0010  0.7315
      5       0.0275        3.7636       0.0550        3.6519     +  0.0010  0.7400
      6       0.0365        3.7282       0.0737        3.6416     +  0.0010  0.7521
      7       0.0357        3.7194       0.0838        3.6302     +  0.0010  0.7642
      8       0.0395        3.6982       0.0975        3.6140     +  0.0010  0.7607
      9       0.0460        3.6795       0.1263        3.5960     +  0.0010  0.7598


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_10_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 10
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 66.2%
subject 24 的准确率: 67.5%
subject 27 的准确率: 72.5%
subject 28 的准确率: 57.5%
subject 31 的准确率: 64.2%
subject 34 的准确率: 59.2%
subject 35 的准确率: 69.6%
所有被试的准确率： {'22': 0.6625, '24': 0.675, '27': 0.725, '28': 0.575, '31': 0.6416666666666667, '34': 0.5916666666666667, '35': 0.6958333333333333}
所有被试的平均准确率: 65.2%
所有被试准确率（无名称）: 66.2,67.5,72.5,57.5,64.2,59.2,69.6


In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 20

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_20_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0280        3.9092       0.0300        3.6861     +  0.0010  1.1381
      2       0.0307        3.7721       0.0413        3.6717     +  0.0010  0.7568
      3       0.0320        3.7329       0.0587        3.6546     +  0.0010  0.7320
      4       0.0370        3.7067       0.0775        3.6365     +  0.0010  0.7624
      5       0.0428        3.6792       0.0850        3.6159     +  0.0010  0.7375
      6       0.0530        3.6553       0.1125        3.5890     +  0.0010  0.7294
      7       0.0565        3.6166       0.1288        3.5565     +  0.0010  0.7419
      8       0.0678        3.5816       0.1537        3.5150     +  0.0010  0.7337
      9       0.0953        3.5295       0.1800        3.4543     +  0.0010  0.7247


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_20_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 20
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.4%
subject 24 的准确率: 78.3%
subject 27 的准确率: 82.9%
subject 28 的准确率: 65.8%
subject 31 的准确率: 73.3%
subject 34 的准确率: 64.6%
subject 35 的准确率: 76.2%
所有被试的准确率： {'22': 0.7541666666666667, '24': 0.7833333333333333, '27': 0.8291666666666667, '28': 0.6583333333333333, '31': 0.7333333333333333, '34': 0.6458333333333334, '35': 0.7625}
所有被试的平均准确率: 73.8%
所有被试准确率（无名称）: 75.4,78.3,82.9,65.8,73.3,64.6,76.2


In [7]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_30_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0275        3.8405       0.0338        3.6851     +  0.0010  0.9831
      2       0.0343        3.7332       0.0462        3.6663     +  0.0010  0.7150
      3       0.0352        3.6962       0.0650        3.6432     +  0.0010  0.7304
      4       0.0423        3.6574       0.0875        3.6116     +  0.0010  0.7334
      5       0.0625        3.6193       0.1075        3.5743     +  0.0010  0.7407
      6       0.0705        3.5789       0.1313        3.5228     +  0.0010  0.7459
      7       0.0980        3.5110       0.1688        3.4499     +  0.0010  0.7148
      8       0.1360        3.4329       0.1900        3.3555     +  0.0010  0.7234
      9       0.1588        3.3537       0.2037        3.2531     +  0.0010  0.7353


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_30_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.4%
subject 24 的准确率: 78.8%
subject 27 的准确率: 82.5%
subject 28 的准确率: 69.6%
subject 31 的准确率: 76.2%
subject 34 的准确率: 64.6%
subject 35 的准确率: 78.8%
所有被试的准确率： {'22': 0.7541666666666667, '24': 0.7875, '27': 0.825, '28': 0.6958333333333333, '31': 0.7625, '34': 0.6458333333333334, '35': 0.7875}
所有被试的平均准确率: 75.1%
所有被试准确率（无名称）: 75.4,78.8,82.5,69.6,76.2,64.6,78.8


In [9]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 40

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_40_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0253        3.7926       0.0362        3.6835     +  0.0010  0.7601
      2       0.0300        3.7154       0.0512        3.6623     +  0.0010  0.7208
      3       0.0393        3.6785       0.0737        3.6362     +  0.0010  0.7247
      4       0.0540        3.6412       0.0950        3.6006     +  0.0010  0.7393
      5       0.0675        3.6038       0.1225        3.5544     +  0.0010  0.7374
      6       0.0833        3.5563       0.1500        3.4907     +  0.0010  0.7409
      7       0.1212        3.4844       0.1875        3.3906     +  0.0010  0.7441
      8       0.1522        3.3982       0.2100        3.2855     +  0.0010  0.7242
      9       0.1822        3.3109       0.2250        3.1778     +  0.0010  0.7224


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_40_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 40
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 74.6%
subject 24 的准确率: 76.7%
subject 27 的准确率: 82.5%
subject 28 的准确率: 71.2%
subject 31 的准确率: 80.0%
subject 34 的准确率: 67.9%
subject 35 的准确率: 80.0%
所有被试的准确率： {'22': 0.7458333333333333, '24': 0.7666666666666667, '27': 0.825, '28': 0.7125, '31': 0.8, '34': 0.6791666666666667, '35': 0.8}
所有被试的平均准确率: 76.1%
所有被试准确率（无名称）: 74.6,76.7,82.5,71.2,80.0,67.9,80.0


In [11]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 50

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_50_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        3.7601       0.0362        3.6839     +  0.0010  1.1283
      2       0.0300        3.7039       0.0525        3.6625     +  0.0010  0.7516
      3       0.0362        3.6751       0.0762        3.6361     +  0.0010  0.7335
      4       0.0535        3.6448       0.0912        3.5999     +  0.0010  0.7378
      5       0.0648        3.6135       0.1175        3.5527     +  0.0010  0.7420
      6       0.0830        3.5722       0.1350        3.4869     +  0.0010  0.7504
      7       0.1207        3.5093       0.1850        3.3830     +  0.0010  0.7419
      8       0.1465        3.4333       0.1963        3.2758     +  0.0010  0.7291
      9       0.1762        3.3537       0.2100        3.1676     +  0.0010  0.7375


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_50_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 50
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.4%
subject 24 的准确率: 75.4%
subject 27 的准确率: 85.4%
subject 28 的准确率: 69.6%
subject 31 的准确率: 78.3%
subject 34 的准确率: 67.5%
subject 35 的准确率: 81.7%
所有被试的准确率： {'22': 0.7541666666666667, '24': 0.7541666666666667, '27': 0.8541666666666666, '28': 0.6958333333333333, '31': 0.7833333333333333, '34': 0.675, '35': 0.8166666666666667}
所有被试的平均准确率: 76.2%
所有被试准确率（无名称）: 75.4,75.4,85.4,69.6,78.3,67.5,81.7


In [13]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 60

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_60_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        3.7413       0.0362        3.6844     +  0.0010  1.0024
      2       0.0302        3.6977       0.0525        3.6630     +  0.0010  0.7573
      3       0.0357        3.6740       0.0750        3.6364     +  0.0010  0.7470
      4       0.0498        3.6486       0.0887        3.6001     +  0.0010  0.7206
      5       0.0645        3.6218       0.1150        3.5523     +  0.0010  0.7235
      6       0.0790        3.5852       0.1313        3.4856     +  0.0010  0.7296
      7       0.1180        3.5295       0.1787        3.3801     +  0.0010  0.7358
      8       0.1403        3.4615       0.1875        3.2726     +  0.0010  0.7440
      9       0.1690        3.3883       0.2025        3.1660     +  0.0010  0.7310


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_60_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 60
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.8%
subject 24 的准确率: 77.9%
subject 27 的准确率: 85.0%
subject 28 的准确率: 68.3%
subject 31 的准确率: 76.7%
subject 34 的准确率: 72.5%
subject 35 的准确率: 79.6%
所有被试的准确率： {'22': 0.7583333333333333, '24': 0.7791666666666667, '27': 0.85, '28': 0.6833333333333333, '31': 0.7666666666666667, '34': 0.725, '35': 0.7958333333333333}
所有被试的平均准确率: 76.5%
所有被试准确率（无名称）: 75.8,77.9,85.0,68.3,76.7,72.5,79.6


In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 70

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_70_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        3.7294       0.0400        3.6848     +  0.0010  0.8316
      2       0.0315        3.6940       0.0550        3.6637     +  0.0010  0.7510
      3       0.0350        3.6739       0.0725        3.6370     +  0.0010  0.7427
      4       0.0498        3.6521       0.0862        3.6007     +  0.0010  0.7591
      5       0.0633        3.6287       0.1087        3.5529     +  0.0010  0.7511
      6       0.0775        3.5960       0.1288        3.4860     +  0.0010  0.7587
      7       0.1148        3.5463       0.1713        3.3805     +  0.0010  0.7543
      8       0.1383        3.4848       0.1862        3.2736     +  0.0010  0.7537
      9       0.1635        3.4169       0.1963        3.1694     +  0.0010  0.7682


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_70_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 70
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 72.5%
subject 24 的准确率: 77.1%
subject 27 的准确率: 81.7%
subject 28 的准确率: 68.3%
subject 31 的准确率: 76.2%
subject 34 的准确率: 68.3%
subject 35 的准确率: 76.7%
所有被试的准确率： {'22': 0.725, '24': 0.7708333333333334, '27': 0.8166666666666667, '28': 0.6833333333333333, '31': 0.7625, '34': 0.6833333333333333, '35': 0.7666666666666667}
所有被试的平均准确率: 74.4%
所有被试准确率（无名称）: 72.5,77.1,81.7,68.3,76.2,68.3,76.7


In [8]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 80

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_80_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0257        3.7213       0.0400        3.6852     +  0.0010  1.0317
      2       0.0315        3.6916       0.0500        3.6643     +  0.0010  0.8213
      3       0.0352        3.6742       0.0700        3.6376     +  0.0010  0.7619
      4       0.0478        3.6552       0.0800        3.6016     +  0.0010  0.7834
      5       0.0590        3.6345       0.1050        3.5539     +  0.0010  0.7553
      6       0.0752        3.6050       0.1237        3.4875     +  0.0010  0.8215
      7       0.1103        3.5602       0.1663        3.3828     +  0.0010  0.7708
      8       0.1350        3.5042       0.1725        3.2767     +  0.0010  0.8378
      9       0.1595        3.4409       0.1913        3.1753     +  0.0010  0.7722


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_80_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 80
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.0%
subject 24 的准确率: 77.9%
subject 27 的准确率: 83.3%
subject 28 的准确率: 65.4%
subject 31 的准确率: 75.4%
subject 34 的准确率: 67.1%
subject 35 的准确率: 76.2%
所有被试的准确率： {'22': 0.75, '24': 0.7791666666666667, '27': 0.8333333333333334, '28': 0.6541666666666667, '31': 0.7541666666666667, '34': 0.6708333333333333, '35': 0.7625}
所有被试的平均准确率: 74.3%
所有被试准确率（无名称）: 75.0,77.9,83.3,65.4,75.4,67.1,76.2


In [10]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 90

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_90_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0250        3.7155       0.0387        3.6856     +  0.0010  0.7463
      2       0.0325        3.6901       0.0488        3.6650     +  0.0010  0.7216
      3       0.0348        3.6746       0.0612        3.6382     +  0.0010  0.7454
      4       0.0462        3.6578       0.0775        3.6024     +  0.0010  0.7422
      5       0.0563        3.6392       0.1062        3.5548     +  0.0010  0.8081
      6       0.0730        3.6122       0.1200        3.4885     +  0.0010  0.8471
      7       0.1087        3.5713       0.1613        3.3839     +  0.0010  0.7365
      8       0.1325        3.5199       0.1713        3.2797     +  0.0010  0.8056
      9       0.1535        3.4605       0.1888        3.1817     +  0.0010  0.7282


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_90_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 90
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 67.5%
subject 24 的准确率: 78.3%
subject 27 的准确率: 79.2%
subject 28 的准确率: 60.0%
subject 31 的准确率: 71.2%
subject 34 的准确率: 65.4%
subject 35 的准确率: 71.7%
所有被试的准确率： {'22': 0.675, '24': 0.7833333333333333, '27': 0.7916666666666666, '28': 0.6, '31': 0.7125, '34': 0.6541666666666667, '35': 0.7166666666666667}
所有被试的平均准确率: 70.5%
所有被试准确率（无名称）: 67.5,78.3,79.2,60.0,71.2,65.4,71.7


In [12]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 100

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_100_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0248        3.7112       0.0375        3.6859     +  0.0010  0.7745
      2       0.0320        3.6890       0.0500        3.6655     +  0.0010  0.7737
      3       0.0350        3.6750       0.0612        3.6388     +  0.0010  0.7906
      4       0.0440        3.6601       0.0775        3.6032     +  0.0010  0.7708
      5       0.0558        3.6432       0.1025        3.5557     +  0.0010  0.7502
      6       0.0722        3.6183       0.1200        3.4898     +  0.0010  0.7673
      7       0.1067        3.5807       0.1600        3.3860     +  0.0010  0.7648
      8       0.1303        3.5331       0.1663        3.2832     +  0.0010  0.7873
      9       0.1482        3.4773       0.1825        3.1886     +  0.0010  0.7435


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_100_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 1.0
epsilon = 100
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 63.7%
subject 24 的准确率: 72.9%
subject 27 的准确率: 75.8%
subject 28 的准确率: 59.6%
subject 31 的准确率: 68.8%
subject 34 的准确率: 62.1%
subject 35 的准确率: 70.8%
所有被试的准确率： {'22': 0.6375, '24': 0.7291666666666666, '27': 0.7583333333333333, '28': 0.5958333333333333, '31': 0.6875, '34': 0.6208333333333333, '35': 0.7083333333333334}
所有被试的平均准确率: 67.7%
所有被试准确率（无名称）: 63.7,72.9,75.8,59.6,68.8,62.1,70.8
